tf.random.normal(
    shape,
    mean=0.0,
    stddev=1.0,
    dtype=tf.dtypes.float32,
    seed=None,
    name=None
) -> Outputs random values from a normal distribution

tf.nn.softmax(
    logits,
    axis=None,
    name=None,
    dim=None
)

tf.math.reduce_sum(
    input_tensor,
    axis=None,
    keepdims=None,
    name=None,
    reduction_indices=None,
    keep_dims=None
) Computes the sum of elements across dimensions of a tensor. 
Reduces input_tensor along the dimensions given in axis

tf.math.reduce_mean(
    input_tensor,
    axis=None,
    keepdims=None,
    name=None,
    reduction_indices=None,
    keep_dims=None
) Computes the mean of elements across dimensions of a tensor.
Reduces input_tensor along the dimensions given in axis

In [3]:
import tensorflow as tf
import numpy as np
import json
import random

import sys
sys.path.insert(0, '../Library')

from helpers import *

In [4]:
###
### Citest json files si creeaza lista de cuvinte unice din venues. Scrie rezultatul in fisier.
### return numarul de cuvinte.
###
def creeaza_lista_cuvinte_unice():
    lista_unica_cuvinte = []
    
    ## Citeste lista de venues
    r = open('../Date/Initiale/lista_venue_completa.txt', 'r')
    lista_venues = json.load(r)
    r.close()
    
    ## Creeaza lista de cuvinte unice din venues
    for venue in lista_venues:
        lista_unica_cuvinte += venue
    lista_unica_cuvinte = set(lista_unica_cuvinte)
    
    w = open('../Date/Initiale/lista_unica_cuvinte_venues.txt','w')
    w.write(json.dumps(list(lista_unica_cuvinte)))
    w.close()

    return len(lista_unica_cuvinte)

###
### Citest json files si creeaza lista de cuvinte unice din abstract. Scrie rezultatul in fisier.
### return numarul de cuvinte.
###
def creeaza_lista_cuvinte_unice_abstract():
    lista_unica_cuvinte = []
    
    ## Citeste lista de abstract
    for i in range(0,169):
        r = open('../Date/Initiale/Complet/file'+str(i)+'.txt', 'r')
        
        for line in r:
            art_crt = json.loads(line)
            lista_unica_cuvinte+=art_crt['abstract']
            
        r.close()
        lista_unica_cuvinte = remove_duplicates(lista_unica_cuvinte)
        
    w = open('../Date/Initiale/lista_unica_cuvinte_abstract.txt','w')
    w.write(json.dumps(list(lista_unica_cuvinte)))
    w.close()

    return len(lista_unica_cuvinte)

###
### Citeste lista de cuvinte unice si creeaza dictionar {cuvant:id}. Scrie rezultatul in fisier.
### param file_lista_cuvinte: calea spre fisierul de unde se citeste lista de cuvinte unice.
### param out_file: calea spre fisierul unde se va scrie rezultatul
###
def creeaza_word2int(file_lista_cuvinte, out_file):
    # Citeste lista cuvinte
    r = open(file_lista_cuvinte,'r')
    lista_unica_cuvinte = json.load(r)
    r.close()
    
    word2int = {}
    
    for i,cuvant in enumerate(lista_unica_cuvinte):
        word2int[cuvant] = i
        
    w = open(out_file,'w')
    w.write(json.dumps(word2int))
    w.close()
    
    
###
### Citeste lista de cuvinte unice si creeaza dictionar {id:cuvant}. Scrie rezultatul in fisier.
### param file_lista_cuvinte: calea spre fisierul de unde se citeste lista de cuvinte unice.
### param out_file: calea spre fisierul unde se va scrie rezultatul
###
def creeaza_int2word(file_lista_cuvinte, out_file):
    # Citeste lista cuvinte
    r = open(file_lista_cuvinte,'r')
    lista_unica_cuvinte = json.load(r)
    r.close()
    
    int2word = {}
    
    for i,cuvant in enumerate(lista_unica_cuvinte):
        int2word[i] = cuvant
        
    w = open(out_file,'w')
    w.write(json.dumps(int2word))
    w.close()

###
### Creeaza un vector de tip one hot.
### param index: pozitia din vector pe care se va pune 1.
### param len_vect: lungimea vectorului.
### return hot_vect: vectorul rezultat.
###
def one_hot(index, len_vect):
    hot_vect = np.zeros(len_vect)
    hot_vect[index] = 1
    return hot_vect

###
###
###
def creeaza_lista_vecini(word2int_file, docs_file, DIM_FEREASTRA, DIM_VOCABULAR):
    # Citeste word2int
    r = open(word2int_file, 'r')
    word2int = json.load(r)
    r.close()
    
    # Citeste lista documente
    r = open(docs_file, 'r')
    lista_doc = json.load(r)
    r.close()
    
    X_train = [] 
    Y_train = [] 
    
    for doc in lista_doc:
        for word_index, word in enumerate(doc):
            for vecin in doc[max(word_index - DIM_FEREASTRA, 0) : min(word_index + DIM_FEREASTRA, len(doc)) + 1] : 
                if vecin != word:
                    X_train.append(one_hot(word2int[ word ], DIM_VOCABULAR))
                    Y_train.append(one_hot(word2int[ vecin ], DIM_VOCABULAR))

    X_train = np.asarray(X_train)
    Y_train = np.asarray(Y_train)
    
    return X_train, Y_train

def log(word2int_file, vectors, file):
    r = open(word2int_file, 'r')
    word2int = json.load(r)
    r.close()
    
    dict_ = {}
    for elem in word2int:
        dict_[elem] = vectors[word2int[elem]].tolist()
    w = open (file, 'w')
    w.write(json.dumps(dict_))
    w.close()       

## Creeaza dictionar cuvinte unice venue

In [5]:
DIM_VOCABULAR = creeaza_lista_cuvinte_unice()
print(DIM_VOCABULAR)
creeaza_word2int('../Date/Initiale/lista_unica_cuvinte_venues.txt','../DATE/word2int_venue.txt')
creeaza_int2word('../Date/Initiale/lista_unica_cuvinte_venues.txt','../DATE/int2word_venue.txt')

1416


## Antrenare

In [21]:
def train (DIM_VOCABULAR, DIM_FEREASTRA, learning_rate, epoci, word2int_path, complete_path):
    X_train, Y_train = creeaza_lista_vecini(word2int_path, complete_path, DIM_FEREASTRA, DIM_VOCABULAR)
    
    #### Creare retea
    X = tf.placeholder(tf.float32, shape=(None, DIM_VOCABULAR))
    Y_eticheta = tf.placeholder(tf.float32, shape=(None, DIM_VOCABULAR))

    ## Din layer de input in hidden layer
    EMBEDDING_DIM = 50
    W1 = tf.Variable(tf.random_normal([DIM_VOCABULAR, EMBEDDING_DIM]))
    b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) #bias
    layer_hidden = tf.add(tf.matmul(X,W1), b1)

    ## Din hidden layer in output layer
    W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, DIM_VOCABULAR]))
    b2 = tf.Variable(tf.random_normal([DIM_VOCABULAR]))
    output_layer = tf.add( tf.matmul(layer_hidden, W2), b2)
    
    #### Instantiere model tensorflow
    with tf.device('/gpu:0'):
        sess = tf.Session()
        init = tf.global_variables_initializer()
        sess.run(init)

        # Functia de loss -> cross entropy
        #cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(Y_eticheta * tf.log(output_layer), reduction_indices=[1]))
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y_eticheta, logits=output_layer))
 
        # compute_gradients + apply_gradients = minimize
        train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

        i = 0
        crt_poz = 0
        end_poz = 0
        print(len(X_train))
        while(1):
            if i >= epoci:
                break
            i+=1 
            end_poz = crt_poz + 100
            if end_poz >= len(X_train):
                end_poz = len(X_train)
            size_mini_batch = end_poz - crt_poz
            
            x_mini_batch = X_train[crt_poz:end_poz].reshape(size_mini_batch,DIM_VOCABULAR) # Transpose to the correct shape
            y_mini_batch = Y_train[crt_poz:end_poz].reshape(size_mini_batch,DIM_VOCABULAR)
            
            
            sess.run(train_step, feed_dict={X: x_mini_batch, Y_eticheta: y_mini_batch})
            l = sess.run(cross_entropy, feed_dict={X: x_mini_batch, Y_eticheta: y_mini_batch})

            w = open('loss.txt','a')
            w.write(str(l))
            w.write(' ')
            w.close()
            print('loss is : ', l, i)
            if end_poz == len(X_train):
                crt_poz = 0
            crt_poz += 2000

    vectors = sess.run(W1+b1)
    return vectors
            
            
            
           

## Antrenare venue

In [43]:
vectors = train (DIM_VOCABULAR, 2, 0.1, 5000, '../Date/Initiale/word2int_venues.txt', '../Date/Initiale/lista_venue_completa.txt')

29582
loss is :  32.713898 1
loss is :  28.283743 2
loss is :  29.497566 3
loss is :  29.88277 4
loss is :  29.941984 5
loss is :  25.486591 6
loss is :  27.465158 7
loss is :  22.183355 8
loss is :  24.906748 9
loss is :  22.0714 10
loss is :  21.52895 11
loss is :  26.273474 12
loss is :  23.039288 13
loss is :  24.999094 14
loss is :  23.759361 15
loss is :  nan 16
loss is :  23.704407 17
loss is :  24.922136 18
loss is :  25.268333 19
loss is :  26.068073 20
loss is :  21.37656 21
loss is :  22.258068 22
loss is :  17.389881 23
loss is :  20.083063 24
loss is :  19.40483 25
loss is :  19.60583 26
loss is :  23.270706 27
loss is :  20.641184 28
loss is :  23.07743 29
loss is :  21.39701 30
loss is :  nan 31
loss is :  22.070328 32
loss is :  23.08125 33
loss is :  23.529469 34
loss is :  24.500381 35
loss is :  19.689241 36
loss is :  20.196455 37
loss is :  15.356066 38
loss is :  17.775724 39
loss is :  18.24504 40
loss is :  18.845613 41
loss is :  21.888535 42
loss is :  19.5066

loss is :  12.176525 343
loss is :  14.887623 344
loss is :  11.85991 345
loss is :  nan 346
loss is :  15.170837 347
loss is :  15.966275 348
loss is :  16.714333 349
loss is :  17.456968 350
loss is :  10.89489 351
loss is :  11.44758 352
loss is :  7.8975453 353
loss is :  8.289394 354
loss is :  11.425387 355
loss is :  12.573756 356
loss is :  13.4087715 357
loss is :  11.945972 358
loss is :  14.673347 359
loss is :  11.597854 360
loss is :  nan 361
loss is :  14.96814 362
loss is :  15.758871 363
loss is :  16.513908 364
loss is :  17.236084 365
loss is :  10.689544 366
loss is :  11.244305 367
loss is :  7.7542267 368
loss is :  8.115909 369
loss is :  11.255724 370
loss is :  12.395566 371
loss is :  13.1320095 372
loss is :  11.724076 373
loss is :  14.465528 374
loss is :  11.344761 375
loss is :  nan 376
loss is :  14.76911 377
loss is :  15.558198 378
loss is :  16.317461 379
loss is :  17.019411 380
loss is :  10.490074 381
loss is :  11.046714 382
loss is :  7.6158686 38

loss is :  12.613662 693
loss is :  12.83368 694
loss is :  13.259519 695
loss is :  7.5795326 696
loss is :  7.9944124 697
loss is :  5.6310616 698
loss is :  5.723437 699
loss is :  8.574992 700
loss is :  9.398665 701
loss is :  8.818991 702
loss is :  8.167279 703
loss is :  11.019252 704
loss is :  7.541462 705
loss is :  nan 706
loss is :  11.07008 707
loss is :  12.492222 708
loss is :  12.695936 709
loss is :  13.110561 710
loss is :  7.481458 711
loss is :  7.885144 712
loss is :  5.5663404 713
loss is :  5.659236 714
loss is :  8.483112 715
loss is :  9.293585 716
loss is :  8.692661 717
loss is :  8.05022 718
loss is :  10.895834 719
loss is :  7.427058 720
loss is :  nan 721
loss is :  10.928685 722
loss is :  12.371696 723
loss is :  12.560585 724
loss is :  12.963993 725
loss is :  7.385559 726
loss is :  7.7789655 727
loss is :  5.5034623 728
loss is :  5.5974016 729
loss is :  8.393279 730
loss is :  9.190453 731
loss is :  8.570582 732
loss is :  7.93568 733
loss is : 

loss is :  6.159635 1033
loss is :  8.7344675 1034
loss is :  5.7094502 1035
loss is :  nan 1036
loss is :  8.512086 1037
loss is :  10.055311 1038
loss is :  10.146701 1039
loss is :  10.336805 1040
loss is :  5.7951603 1041
loss is :  6.2103963 1042
loss is :  4.4753103 1043
loss is :  4.60719 1044
loss is :  6.8666935 1045
loss is :  7.362187 1046
loss is :  6.7002087 1047
loss is :  6.094215 1048
loss is :  8.646461 1049
loss is :  5.6492352 1050
loss is :  nan 1051
loss is :  8.416803 1052
loss is :  9.955929 1053
loss is :  10.049098 1054
loss is :  10.230145 1055
loss is :  5.734927 1056
loss is :  6.1543055 1057
loss is :  4.43626 1058
loss is :  4.569204 1059
loss is :  6.8061028 1060
loss is :  7.288711 1061
loss is :  6.629207 1062
loss is :  6.030601 1063
loss is :  8.559414 1064
loss is :  5.5902996 1065
loss is :  nan 1066
loss is :  8.322825 1067
loss is :  9.857513 1068
loss is :  9.952868 1069
loss is :  10.124903 1070
loss is :  5.6758113 1071
loss is :  6.09913 1072


loss is :  3.881436 1374
loss is :  5.6859827 1375
loss is :  5.9882007 1376
loss is :  5.416071 1377
loss is :  5.07407 1378
loss is :  6.93296 1379
loss is :  4.581936 1380
loss is :  nan 1381
loss is :  6.61397 1382
loss is :  8.0120325 1383
loss is :  8.196903 1384
loss is :  8.2069845 1385
loss is :  4.6717863 1386
loss is :  5.1289887 1387
loss is :  3.7309844 1388
loss is :  3.8532562 1389
loss is :  5.639392 1390
loss is :  5.936889 1391
loss is :  5.369005 1392
loss is :  5.0402746 1393
loss is :  6.864889 1394
loss is :  4.5430365 1395
loss is :  nan 1396
loss is :  6.543688 1397
loss is :  7.9346375 1398
loss is :  8.123167 1399
loss is :  8.128858 1400
loss is :  4.6344795 1401
loss is :  5.0910683 1402
loss is :  3.7056704 1403
loss is :  3.825529 1404
loss is :  5.5934014 1405
loss is :  5.886425 1406
loss is :  5.322678 1407
loss is :  5.007089 1408
loss is :  6.7976727 1409
loss is :  4.504881 1410
loss is :  nan 1411
loss is :  6.474337 1412
loss is :  7.8581257 1413
l

...

loss is :  2.6529598 3036
loss is :  2.9732466 3037
loss is :  2.3920646 3038
loss is :  2.5171669 3039
loss is :  3.045725 3040
loss is :  3.0721796 3041
loss is :  2.7358627 3042
loss is :  2.7926157 3043
loss is :  2.8734872 3044
loss is :  2.7415814 3045
loss is :  nan 3046
loss is :  2.6007133 3047
loss is :  2.8400774 3048
loss is :  3.4203753 3049
loss is :  3.3031967 3050
loss is :  2.6448967 3051
loss is :  2.9634337 3052
loss is :  2.386422 3053
loss is :  2.5128937 3054
loss is :  3.0362625 3055
loss is :  3.061509 3056
loss is :  2.7246728 3057
loss is :  2.7805066 3058
loss is :  2.8555164 3059
loss is :  2.7358286 3060
loss is :  nan 3061
loss is :  2.5863695 3062
loss is :  2.820868 3063
loss is :  3.3979058 3064
loss is :  3.279888 3065
loss is :  2.6369894 3066
loss is :  2.9537377 3067
loss is :  2.380847 3068
loss is :  2.5086875 3069
loss is :  3.0269687 3070
loss is :  3.0509706 3071
loss is :  2.7136436 3072
loss is :  2.7685816 3073
loss is :  2.8377857 3074
loss

loss is :  2.5235353 3372
loss is :  2.568077 3373
loss is :  2.5297287 3374
loss is :  2.6307206 3375
loss is :  nan 3376
loss is :  2.324362 3377
loss is :  2.4793668 3378
loss is :  2.9704514 3379
loss is :  2.8507617 3380
loss is :  2.5028455 3381
loss is :  2.7760108 3382
loss is :  2.2779493 3383
loss is :  2.4334426 3384
loss is :  2.8667855 3385
loss is :  2.8571432 3386
loss is :  2.5154824 3387
loss is :  2.5598464 3388
loss is :  2.5164552 3389
loss is :  2.626249 3390
loss is :  nan 3391
loss is :  2.313576 3392
loss is :  2.4656932 3393
loss is :  2.9520469 3394
loss is :  2.833122 3395
loss is :  2.4977806 3396
loss is :  2.768731 3397
loss is :  2.2736802 3398
loss is :  2.4303722 3399
loss is :  2.8605616 3400
loss is :  2.8491395 3401
loss is :  2.5075698 3402
loss is :  2.5517716 3403
loss is :  2.503367 3404
loss is :  2.6218116 3405
loss is :  nan 3406
loss is :  2.3029344 3407
loss is :  2.4522307 3408
loss is :  2.9338064 3409
loss is :  2.8157308 3410
loss is :  

loss is :  2.6378117 3712
loss is :  2.196976 3713
loss is :  2.3735185 3714
loss is :  2.7482378 3715
loss is :  2.7031312 3716
loss is :  2.3732042 3717
loss is :  2.4137838 3718
loss is :  2.2679703 3719
loss is :  2.5353904 3720
loss is :  nan 3721
loss is :  2.1132886 3722
loss is :  2.2126143 3723
loss is :  2.5863264 3724
loss is :  2.5070603 3725
loss is :  2.4074028 3726
loss is :  2.6325495 3727
loss is :  2.193944 3728
loss is :  2.3711267 3729
loss is :  2.7435448 3730
loss is :  2.6970785 3731
loss is :  2.368177 3732
loss is :  2.4085097 3733
loss is :  2.2585223 3734
loss is :  2.531574 3735
loss is :  nan 3736
loss is :  2.105864 3737
loss is :  2.2029912 3738
loss is :  2.5714011 3739
loss is :  2.4949648 3740
loss is :  2.404042 3741
loss is :  2.6273704 3742
loss is :  2.1909685 3743
loss is :  2.3687613 3744
loss is :  2.7388964 3745
loss is :  2.6910923 3746
loss is :  2.3632534 3747
loss is :  2.4033358 3748
loss is :  2.2492275 3749
loss is :  2.527785 3750
loss 

loss is :  1.9027501 4368
loss is :  2.0969093 4369
loss is :  2.1364727 4370
loss is :  2.3023949 4371
loss is :  2.4747312 4372
loss is :  2.1089883 4373
loss is :  2.2898743 4374
loss is :  2.5723786 4375
loss is :  2.4843376 4376
loss is :  2.228827 4377
loss is :  2.258214 4378
loss is :  1.9829675 4379
loss is :  2.399009 4380
loss is :  nan 4381
loss is :  1.8887496 4382
loss is :  1.8975412 4383
loss is :  2.0894837 4384
loss is :  2.1304286 4385
loss is :  2.300784 4386
loss is :  2.472295 4387
loss is :  2.107798 4388
loss is :  2.288447 4389
loss is :  2.5689933 4390
loss is :  2.4803257 4391
loss is :  2.2269967 4392
loss is :  2.2560606 4393
loss is :  1.9790608 4394
loss is :  2.3967633 4395
loss is :  nan 4396
loss is :  1.8851802 4397
loss is :  1.8924011 4398
loss is :  2.0822082 4399
loss is :  2.1244724 4400
loss is :  2.2992008 4401
loss is :  2.4698966 4402
loss is :  2.1066306 4403
loss is :  2.2870407 4404
loss is :  2.5656343 4405
loss is :  2.476355 4406
loss i

loss is :  2.1966193 4707
loss is :  2.2188199 4708
loss is :  1.9141817 4709
loss is :  2.3562474 4710
loss is :  nan 4711
loss is :  1.8185756 4712
loss is :  1.8000256 4713
loss is :  1.9559366 4714
loss is :  2.0184994 4715
loss is :  2.2706814 4716
loss is :  2.4264703 4717
loss is :  2.086592 4718
loss is :  2.2621648 4719
loss is :  2.5015318 4720
loss is :  2.4025326 4721
loss is :  2.1954603 4722
loss is :  2.21734 4723
loss is :  1.9117746 4724
loss is :  2.3545716 4725
loss is :  nan 4726
loss is :  1.815798 4727
loss is :  1.7963691 4728
loss is :  1.9508859 4729
loss is :  2.0142808 4730
loss is :  2.269496 4731
loss is :  2.4246752 4732
loss is :  2.085816 4733
loss is :  2.2611806 4734
loss is :  2.498786 4735
loss is :  2.399458 4736
loss is :  2.1943216 4737
loss is :  2.2158806 4738
loss is :  1.9094182 4739
loss is :  2.3529139 4740
loss is :  nan 4741
loss is :  1.8130579 4742
loss is :  1.7927796 4743
loss is :  1.9459023 4744
loss is :  2.0101285 4745
loss is :  2

In [44]:
f_log = 'model_tf_venue_2_01.txt'
log('../Date/Initiale/word2int_venues.txt', vectors, f_log )